# Why is Home advantage even a thing?

I have never understood how simply playing in a different venue can change the outcome of a sport event. Does this effect even exist? If so, how strong it is? and what is the root cause?

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')
%matplotlib inline

from subprocess import check_output
print(check_output(["ls", "../input"]).decode("utf8"))

Countries = pd.read_csv("../input/Country_facts.csv")
NBA = pd.read_csv("../input/NBA.csv", encoding = "ISO-8859-1") 
Soccer = pd.read_csv("../input/SoccerLEagues.csv",encoding = "ISO-8859-1")

# Let's start with the NBA

Let's produce a scatter plot of the away winning percentage vs the home winning percentage. the further below the diagonal line the points are, the more significant the home advantage is.

In [ ]:
plt.plot(NBA.HomePCT + np.random.normal(0,0.01,len(NBA)),NBA.AwayPCT + np.random.normal(0,0.01,len(NBA)),'o', alpha = 0.4)
plt.plot([0,1],[0,1])
plt.xlabel('Home PCT')
plt.ylabel('Away PCT')
plt.title('Home vs Away PCT in the NBA')

Wow. of 1216 NBA teams over the last 50 years, only 16 teams performed away better than at home.

# Home advantage is ***Definitely*** a thing


In [ ]:
Soccer['HomePCT'] = np.true_divide(Soccer['HomeWins'],Soccer['HomeWins'] + Soccer['HomeLoss'] + Soccer['HomeDraw'])
Soccer['AwayPCT'] = np.true_divide(Soccer['AwayWins'],Soccer['AwayWins'] + Soccer['AwayLoss'] + Soccer['AwayDraw'])

# Let's see how things look in soccer

In [ ]:
plt.plot(Soccer.HomePCT + np.random.normal(0,0.01,len(Soccer)),Soccer.AwayPCT + np.random.normal(0,0.01,len(Soccer)),'o', alpha = 0.1)
plt.plot([0,1],[0,1])
plt.xlabel('Home PCT')
plt.ylabel('Away PCT')
plt.title('Home vs Away PCT in Soccer Leagues')
plt.xlim([0,1])
plt.ylim([0,1])

The home advantage is very noticeable in soccer too.

However, not as much as in the NBA. about 15% of the teams had a higher winning percent away than at home.

This can be (and probably is) due to the fact that a NBA season is 82 games long, which makes it less likely for a team to keep this anomaly consistently over a full year.

# So - in which sport the home advantage is more significant?

Let's define the home factor as following: (Home PCT - Away PCT)/(Home PCT + Away PCT)

The values would range from -1 to 1, where 1 implies maximal home advantage, and the opposite for -1.


In [ ]:
NBA['Home_Factor'] = np.true_divide((NBA.HomePCT - NBA.AwayPCT),(NBA.HomePCT + NBA.AwayPCT))
Soccer['Home_Factor'] = np.true_divide((Soccer.HomePCT - Soccer.AwayPCT),(Soccer.HomePCT + Soccer.AwayPCT))

In [ ]:
import seaborn as sns
sns.kdeplot(Soccer.Home_Factor, shade = True)
sns.kdeplot(NBA.Home_Factor, shade = True)
plt.xlim([-0.8,0.8])
plt.legend(['Soccer Leagues','NBA'])
plt.plot([0,0],[0,3])
plt.xlabel('Home Factor')

While the soccer histogram is wider - again probably due to the fact the soccer seasons are much shorter, allowing for more statistical out-liars (as well as single soccer matches who are arguably more random) - the mean is effectively the same!

This is a bit weird. If home advantage depends on things like referees, long bus rides, hostile audience, etc., I would except the home advantage to behave very differently in American Basketball and worldwide soccer. but it doesn't implying that the home advantage may stem from deeper and more basic human behavior. 

# Let's see how the goals difference changes between home and away

In [ ]:
plt.style.use('fivethirtyeight')
plt.plot(Soccer.HomeRatio + np.random.normal(0,0.5,len(Soccer)),Soccer.AwayGoalsDiff+ np.random.normal(0,0.5,len(Soccer)),'o', alpha = 0.2)
plt.plot([-100,80],[-100,80])
plt.xlim([-40,40])
plt.ylim([-40,40])
plt.xlabel('Home Goals Difference')
plt.ylabel('Away Goals Difference')

# Has the effect changed over time?

Since we have NBA data for a few decades, let see if it has developed over time

In [ ]:
plt.plot(NBA.Year + np.random.normal(0,0.5,len(NBA)),NBA.Home_Factor,'o', alpha = 0.8)
x = NBA.Year
y = NBA.Home_Factor
fn = np.polyfit(x,y,1)
fit_fn = np.poly1d(fn) 
plt.plot(x,fit_fn(x))
plt.xlabel('Year')
plt.ylim([-1,1])
plt.xlim([1950,2010])

It seems like there's a modest decline in home advantage in the last decades

Let's see what other factors can affect this phenomenon. It is conventional wisdom that one of the causes is the home crowd who pushes the home team and frighting or annoying the away team, but is it really the case? it is difficult to test it since we don't have the crowd size in each and every game, but we due have the average league attendance.

In [ ]:

plt.figure(figsize = (10,10))
plt.subplot(2,1,1)
plt.title('Home Advantage vs FIFA Rank')
plt.plot(Countries.FIFA_Rank, Countries.Home_Away_Contrast,'o')
x = Countries.FIFA_Rank
y = Countries.Home_Away_Contrast
fn = np.polyfit(x,y,1)
fit_fn = np.poly1d(fn) 
plt.plot(x,fit_fn(x))
plt.xlabel('FIFA Rank')

plt.subplot(2,1,2)
plt.title('Home Advantage vs Crowd Attendance')
plt.plot(Countries.Attendance, Countries.Home_Away_Contrast,'o')
x = Countries.Attendance[Countries.Attendance>0]
y = Countries.Home_Away_Contrast[Countries.Attendance>0]
fn = np.polyfit(x,y,1)
fit_fn = np.poly1d(fn) 
plt.plot(x,fit_fn(x))
plt.xlabel('Average Attandance per game in the leagure')

Not only home advantage didn't go up with the average attendance, but it was actually the other way around. Also, we see that the home advantage does not depend on the FIFA rank of the country.

Let's explore more general correlations which are not necessarily directly connected so sports and soccer.

In [ ]:
plt.figure(figsize = (10,10))
plt.subplot(2,2,1)
x = Countries.Phones[Countries.Phones>0]
y = Countries.Home_Away_Contrast[Countries.Phones>0]
fn = np.polyfit(x,y,1)
fit_fn = np.poly1d(fn) 
plt.plot(x,y,'o')
plt.plot(x,fit_fn(x))
plt.title('# of Phones per person')

plt.subplot(2,2,2)
x = Countries.GDP[Countries.GDP>0]
y = Countries.Home_Away_Contrast[Countries.GDP>0]
fn = np.polyfit(x,y,1)
fit_fn = np.poly1d(fn) 
plt.plot(x,y,'o')
plt.plot(x,fit_fn(x))
plt.title('GDP')

plt.subplot(2,2,3)
x = Countries.Agriculture[Countries.Agriculture>0]
y = Countries.Home_Away_Contrast[Countries.Agriculture>0]
fn = np.polyfit(x,y,1)
fit_fn = np.poly1d(fn) 
plt.plot(x,y,'o')
plt.plot(x,fit_fn(x))
plt.title('Agriculture')

plt.subplot(2,2,4)
x = Countries.Literacy[Countries.Literacy>0]
y = Countries.Home_Away_Contrast[Countries.Literacy>0]
fn = np.polyfit(x,y,1)
fit_fn = np.poly1d(fn) 
plt.plot(x,y,'o')
plt.plot(x,fit_fn(x))
plt.title('Literacy')

Interestingly enough, it seems that by far the strongest predictors are indicators of general development like number of phones per person, literacy and GDP.

The more developed the country, the less significant the home advantage (in soccer at least) is!